<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/node_postprocessor/rankGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>
# RankGPT 重新排名演示（梵高维基）

此演示将[RankGPT](https://github.com/sunnweiwei/RankGPT)集成到LlamaIndex中作为重新排名器。

论文：[ChatGPT在搜索中表现如何？调查大型语言模型作为重新排名代理](https://arxiv.org/abs/2304.09542)

`RankGPT`的理念：
* 它是使用LLM（ChatGPT或GPT-4或其他LLM）进行零-shot列表式段落重新排名
* 它应用排列生成方法和滑动窗口策略来高效重新排名段落。

在这个例子中，我们以梵高的维基百科作为示例，比较了使用/不使用RankGPT重新排名的检索结果。
我们展示了两个RankGPT模型：
* OpenAI的`GPT3.5`
* `Mistral`模型。


In [ ]:
%pip install llama-index-postprocessor-rankgpt-rerank
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-openai
%pip install llama-index-llms-ollama

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
import os

OPENAI_API_KEY = "sk-"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

```python
import pandas as pd

# 读取数据
data = pd.read_csv('data.csv')

# 显示数据的前几行
data.head()
```


In [ ]:
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

### 从维基百科下载梵高的维基页面

您可以使用以下Python代码从维基百科下载梵高的维基页面。


In [ ]:
from pathlib import Path
import requests

wiki_titles = [
    "Vincent van Gogh",
]


data_path = Path("data_wiki")

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [ ]:
# 加载文档documents = SimpleDirectoryReader("./data_wiki/").load_data()

### 为这个维基百科页面构建向量存储索引


In [ ]:
index = VectorStoreIndex.from_documents(
    documents,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## 检索 + RankGPT 重新排名
步骤：
1. 设置检索器和重新排名器（作为一个选项）
2. 使用搜索查询检索结果，不进行重新排名
3. 使用启用了RankGPT重新排名的搜索查询检索结果
4. 比较有和没有重新排名的结果


In [ ]:
from llama_index.core.retrievers import VectorIndexRetrieverfrom llama_index.core import QueryBundlefrom llama_index.postprocessor.rankgpt_rerank import RankGPTRerankimport pandas as pdfrom IPython.display import display, HTMLdef get_retrieved_nodes(    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False):    query_bundle = QueryBundle(query_str)    # 配置检索器    retriever = VectorIndexRetriever(        index=index,        similarity_top_k=vector_top_k,    )    retrieved_nodes = retriever.retrieve(query_bundle)    if with_reranker:        # 配置重新排序器        reranker = RankGPTRerank(            llm=OpenAI(                model="gpt-3.5-turbo-16k",                temperature=0.0,                api_key=OPENAI_API_KEY,            ),            top_n=reranker_top_n,            verbose=True,        )        retrieved_nodes = reranker.postprocess_nodes(            retrieved_nodes, query_bundle        )    return retrieved_nodesdef pretty_print(df):    return display(HTML(df.to_html().replace("\\n", "<br>")))def visualize_retrieved_nodes(nodes) -> None:    result_dicts = []    for node in nodes:        result_dict = {"Score": node.score, "Text": node.node.get_text()}        result_dicts.append(result_dict)    pretty_print(pd.DataFrame(result_dicts))

### 无需重新排序的检索前3个结果


In [ ]:
new_nodes = get_retrieved_nodes(
    "Which date did Paul Gauguin arrive in Arles?",
    vector_top_k=3,
    with_reranker=False,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


### 期望的结果是：
```在梵高的苦苦哀求之后，高更于10月23日抵达阿尔勒，在11月，两人一起绘画。高更在他的《向日葵画家》中描绘了梵高。```


In [ ]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.857523,"Gauguin fled Arles, never to see Van Gogh again. They continued to correspond, and in 1890, Gauguin proposed they form a studio in Antwerp. Meanwhile, other visitors to the hospital included Marie Ginoux and Roulin.Despite a pessimistic diagnosis, Van Gogh recovered and returned to the Yellow House on 7 January 1889. He spent the following month between hospital and home, suffering from hallucinations and delusions of poisoning. In March, the police closed his house after a petition by 30 townspeople (including the Ginoux family) who described him as le fou roux ""the redheaded madman""; Van Gogh returned to hospital. Paul Signac visited him twice in March; in April, Van Gogh moved into rooms owned by Dr Rey after floods damaged paintings in his own home. Two months later, he left Arles and voluntarily entered an asylum in Saint-Rémy-de-Provence. Around this time, he wrote, ""Sometimes moods of indescribable anguish, sometimes moments when the veil of time and fatality of circumstances seemed to be torn apart for an instant.""Van Gogh gave his 1889 Portrait of Doctor Félix Rey to Dr Rey. The physician was not fond of the painting and used it to repair a chicken coop, then gave it away. In 2016, the portrait was housed at the Pushkin Museum of Fine Arts and estimated to be worth over $50 million.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Saint-Rémy (May 1889 – May 1890) ====Van Gogh entered the Saint-Paul-de-Mausole asylum on 8 May 1889, accompanied by his caregiver, Frédéric Salles, a Protestant clergyman. Saint-Paul was a former monastery in Saint-Rémy, located less than 30 kilometres (19 mi) from Arles, and it was run by a former naval doctor, Théophile Peyron. Van Gogh had two cells with barred windows, one of which he used as a studio. The clinic and its garden became the main subjects of his paintings."
1,0.853599,"When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realize his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers.""When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh.Van Gogh and Gauguin visited Montpellier in December 1888, where they saw works by Courbet and Delacroix in the Musée Fabre. Their relationship began to deteriorate; Van Gogh adm

#### 发现：正确结果在不重新排名的情况下排在第二位


### 使用RankGPT检索和重新排名前10个结果，并返回前3个


In [ ]:
new_nodes = get_retrieved_nodes(
    "Which date did Paul Gauguin arrive in Arles ?",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
After Reranking, new rank list for nodes: [1, 6, 0, 2, 7, 9, 4, 5, 3, 8]

In [ ]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.852371,"When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realize his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers.""When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh.Van Gogh and Gauguin visited Montpellier in December 1888, where they saw works by Courbet and Delacroix in the Musée Fabre. Their relationship began to deteriorate; Van Gogh admired Gauguin and wanted to be treated as his equal, but Gauguin was arrogant and domineering, which frustrated Van Gogh. They often quarrelled; Van Gogh increasingly feared that Gauguin was going to desert him, and the situation, which Van Gogh described as one of ""excessive tension"", rapidly headed towards crisis point."
1,0.819758,"==== Hospital in Arles (December 1888) ====The exact sequence that led to the mutilation of van Gogh's ear is not known. Gauguin said, fifteen years later, that the night followed several instances of physically threatening behaviour. Their relationship was complex and Theo may have owed money to Gauguin, who suspected the brothers were exploiting him financially. It seems likely that Vincent realised that Gauguin was planning to leave. The following days saw heavy rain, leading to the two men being shut in the Yellow House. Gauguin recalled that Van Gogh followed him after he left for a walk and ""rushed towards me, an open razor in his hand."" This account is uncorroborated; Gauguin was almost certainly absent from the Yellow House that night, most likely staying in a hotel.After an altercation on the evening of 23 December 1888, Van Gogh returned to his room where he seemingly heard voices and either wholly or in part severed his left ear with a razor causing severe bleeding. He bandaged the wound, wrapped the ear in paper and delivered the package to a woman at a brothel Van Gogh and Gauguin both frequented. Van Gogh was found unconscious the next morning by a policeman and taken to hospital, where he was treated by Félix Rey, a young doctor still in training. The ear was brought to the hospital, but Rey did not attempt to reattach it as too much time had passed. Van Gogh researcher and art historian Bernadette Murphy discovered the true identity of the woman named Gabrielle, who died in Arles at the age of 80 in 1952, and

#### 结论：在RankGPT重新排名后，第一个结果是包含答案的正确文本


## 使用其他LLM进行RankGPT重新排序

在RankGPT中，我们可以使用其他语言模型（LLM）来对生成的候选项进行重新排序。这种方法可以帮助我们提高生成结果的质量，使其更符合特定任务的需求。

### 步骤

1. **准备候选项**：首先，我们需要生成一组候选项，可以使用RankGPT或其他生成模型来完成。

2. **使用其他LLM进行重新排序**：接下来，我们可以使用其他语言模型来对生成的候选项进行重新排序。这可以通过计算每个候选项的概率分数来实现，然后根据这些分数对候选项进行排序。

3. **选择排名最高的候选项**：最后，我们可以选择重新排序后排名最高的候选项作为最终的生成结果。

通过这种方法，我们可以利用其他语言模型的能力来提高RankGPT生成结果的质量，从而更好地满足特定任务的需求。


### 使用`Ollama`来为本地的`Mistral`模型提供服务


In [ ]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="mistral", request_timeout=30.0)

In [ ]:
from llama_index.core.retrievers import VectorIndexRetrieverfrom llama_index.core import QueryBundleimport pandas as pdfrom IPython.display import display, HTMLfrom llama_index.llms.huggingface import (    HuggingFaceInferenceAPI,    HuggingFaceLLM,)from llama_index.postprocessor.rankgpt_rerank import RankGPTRerankdef get_retrieved_nodes(    query_str, vector_top_k=5, reranker_top_n=3, with_reranker=False):    query_bundle = QueryBundle(query_str)    # 配置检索器    retriever = VectorIndexRetriever(        index=index,        similarity_top_k=vector_top_k,    )    retrieved_nodes = retriever.retrieve(query_bundle)    if with_reranker:        # 配置重新排序器        reranker = RankGPTRerank(            llm=llm,            top_n=reranker_top_n,            verbose=True,        )        retrieved_nodes = reranker.postprocess_nodes(            retrieved_nodes, query_bundle        )    return retrieved_nodes

In [ ]:
new_nodes = get_retrieved_nodes(
    "Which date did Paul Gauguin arrive in Arles ?",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
After Reranking, new rank list for nodes: [4, 5, 0, 1, 2, 3, 6, 7, 8, 9]

In [ ]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.824605,"He adopted elements of Pointillism, a technique in which a multitude of small coloured dots are applied to the canvas so that when seen from a distance they create an optical blend of hues. The style stresses the ability of complementary colours – including blue and orange – to form vibrant contrasts.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tWhile in Asnières Van Gogh painted parks, restaurants and the Seine, including Bridges across the Seine at Asnières. In November 1887, Theo and Vincent befriended Paul Gauguin who had just arrived in Paris. Towards the end of the year, Vincent arranged an exhibition alongside Bernard, Anquetin, and probably Toulouse-Lautrec, at the Grand-Bouillon Restaurant du Chalet, 43 avenue de Clichy, Montmartre. In a contemporary account, Bernard wrote that the exhibition was ahead of anything else in Paris. There, Bernard and Anquetin sold their first paintings, and Van Gogh exchanged work with Gauguin. Discussions on art, artists, and their social situations started during this exhibition, continued and expanded to include visitors to the show, like Camille Pissarro and his son Lucien, Signac and Seurat. In February 1888, feeling worn out from life in Paris, Van Gogh left, having painted more than 200 paintings during his two years there. Hours before his departure, accompanied by Theo, he paid his first and only visit to Seurat in his studio.=== Artistic breakthrough ======= Arles (1888–89) ====Ill from drink and suffering from smoker's cough, in February 1888 Van Gogh sought refuge in Arles. He seems to have moved with thoughts of founding an art colony. The Danish artist Christian Mourier-Petersen became his companion for two months, and, at first, Arles appeared exotic. In a letter, he described it as a foreign country: ""The Zouaves, the brothels, the adorable little Arlésienne going to her First Communion, the priest in his surplice, who looks like a dangerous rhinoceros, the people drinking absinthe, all seem to me creatures from another world.""The time in Arles became one of Van Gogh's more prolific periods: he completed 200 paintings and more than 100 drawings and watercolours."
1,0.822903,"Two years later, Vincent and Theo paid for the publication of a book on Monticelli paintings, and Vincent bought some of Monticelli's works to add to his collection.Van Gogh learned about Fernand Cormon's atelier from Theo. He worked at the studio in April and May 1886, where he frequented the circle of the Australian artist John Russell, who painted his portrait in 1886. Van Gogh also met fellow students Émile Bernard, Louis Anquetin and Henri de Toulouse-Lautrec – who painted a portrait of him in pastel. They met at Julien ""Père"" Tanguy's paint shop, (which was, at that time, the only place where Paul Cézanne's paintings were displayed). In 1886, two large exhibitions were staged there, showing Pointillism and Neo-impressionism for the first time and bringing attention to Georges Seurat and Paul Signac. Theo kept a stock of Impressionist paintings in his gallery on boulevard Montmartre, but Van Gogh was slow to acknowledge the new developments in art.Conflicts arose between the brothers. At the end of 1886 Theo found living with Vincent to be ""almost unbearable"". By early 1887, they were again at peace, and Vincent had moved to Asnières, a northwestern suburb of Paris, where he got to know Signac. He adopted elements of Pointillism, a technique in which a multitude of small coloured dots are applied to the canvas so that when seen from a distance they create an optical blend of hues. The style stresses the ability of complementary colours – including blue and orange – to form vibrant contrasts.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tWhile in Asnières Van Gogh painted parks, restaurants and the Seine, including Bridges across the Seine at Asnières. In November 1887, Theo and Vincent befriended Paul Gauguin who had just arrived in Paris. Towards 